In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout
import plotly.graph_objects as go
from sklearn.metrics import mean_squared_error

class TickerPrice:
    """Handles loading and filtering stock price data."""
    
    def __init__(self, data_path='data/processed/de_share_prices_processed.csv'):
        self.data_path = data_path

    def get_share_prices(self, tickers=None, start_date=None, end_date=None):
        """Loads and filters share prices."""
        try:
            df = pd.read_csv(self.data_path, parse_dates=['Date'], index_col='Date')
            df.index = pd.to_datetime(df.index)

            if tickers:
                df = df[df['Ticker'].isin(tickers)]
            if start_date:
                df = df[df.index >= pd.to_datetime(start_date)]
            if end_date:
                df = df[df.index <= pd.to_datetime(end_date)]

            return df
        except Exception as e:
            print(f"Error loading data: {e}")
            return None

class StockPricePredictor:
    """Handles LSTM-based stock price prediction."""
    
    def __init__(self, data, ticker):
        self.data = data
        self.ticker = ticker
        self.scaler = MinMaxScaler(feature_range=(0, 1))
        self.model = None

    def preprocess_data(self, sequence_length=50):
        """Prepares data for training and testing."""
        close_prices = self.data[['Close']].values
        scaled_data = self.scaler.fit_transform(close_prices)

        X, y = [], []
        for i in range(sequence_length, len(scaled_data)):
            X.append(scaled_data[i-sequence_length:i, 0])
            y.append(scaled_data[i, 0])

        X, y = np.array(X), np.array(y)
        train_size = int(len(X) * 0.70)
        test_size = int(len(X) * 0.20)

        X_train, y_train = X[:train_size], y[:train_size]
        X_test, y_test = X[train_size:train_size + test_size], y[train_size:train_size + test_size]
        X_val, y_val = X[train_size + test_size:], y[train_size + test_size:]

        X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
        X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
        X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))

        return X_train, y_train, X_test, y_test, X_val, y_val

    def build_model(self):
        """Builds the LSTM model."""
        model = Sequential([
            LSTM(50, return_sequences=True, input_shape=(50, 1)),
            Dropout(0.2),
            LSTM(50, return_sequences=False),
            Dropout(0.2),
            Dense(1)
        ])
        model.compile(optimizer='adam', loss='mean_squared_error')
        self.model = model

    def train_model(self, X_train, y_train, X_val, y_val, epochs=20, batch_size=32):
        """Trains the model."""
        if self.model is None:
            self.build_model()

        self.model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val))

    def save_model(self):
        """Saves the trained model with the ticker name in the filename."""
        if self.model:
            model_dir = "models"
            os.makedirs(model_dir, exist_ok=True)
            model_path = os.path.join(model_dir, f"lstm_model_{self.ticker}.h5")
            self.model.save(model_path)
            print(f"Model saved to {model_path}")
        else:
            print("No trained model found.")

    def load_model(self):
        """Loads a pre-trained model."""
        model_path = f"models/lstm_model_{self.ticker}.h5"
        if os.path.exists(model_path):
            self.model = load_model(model_path)
            print(f"Model loaded from {model_path}")
        else:
            print(f"No model found for {self.ticker}.")

    def predict_next_day(self):
        """Predicts the next day's closing price."""
        last_50_days = self.data['Close'].values[-50:].reshape(-1, 1)
        last_50_scaled = self.scaler.transform(last_50_days).reshape(1, 50, 1)

        next_pred_scaled = self.model.predict(last_50_scaled)[0, 0]
        next_pred = self.scaler.inverse_transform([[next_pred_scaled]])[0, 0]

        return next_pred

    def predict_multiple_days(self, days=2):
        """Predicts multiple days ahead."""
        predictions = []
        last_50_days = self.data['Close'].values[-50:].reshape(-1, 1)
        last_50_scaled = self.scaler.transform(last_50_days)

        for _ in range(days):
            input_data = last_50_scaled.reshape(1, 50, 1)
            next_pred_scaled = self.model.predict(input_data)[0, 0]
            next_pred = self.scaler.inverse_transform([[next_pred_scaled]])[0, 0]
            predictions.append(next_pred)

            next_pred_scaled_array = self.scaler.transform([[next_pred]])
            last_50_scaled = np.append(last_50_scaled[1:], next_pred_scaled_array).reshape(-1, 1)

        return predictions

    def plot_predictions(self, y_true, y_pred, title="Stock Price Predictions"):
        """Plots actual vs predicted prices using Plotly."""
        fig = go.Figure()

        fig.add_trace(go.Scatter(y=y_true, mode='lines', name="Actual Prices", line=dict(color='blue')))
        fig.add_trace(go.Scatter(y=y_pred, mode='lines', name="Predicted Prices", line=dict(color='red', dash='dash')))

        fig.update_layout(title=title, xaxis_title="Time", yaxis_title="Stock Price", legend_title="Legend", template="plotly_white")
        fig.show()

    def evaluate_model(self, X_test, y_test):
        """Evaluates model performance using MSE and RMSE."""
        if self.model is None:
            print("No trained model found.")
            return

        # Make predictions
        y_pred_scaled = self.model.predict(X_test)

        # Inverse transform predictions and actual values
        y_pred = self.scaler.inverse_transform(y_pred_scaled)
        y_test = self.scaler.inverse_transform(y_test.reshape(-1, 1))

        # Compute evaluation metrics
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)

        print(f"Model Evaluation for {self.ticker}:")
        print(f"Mean Squared Error (MSE): {mse:.4f}")
        print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")

class StockModelManager:
    """Manages training and saving models for multiple tickers."""
    
    def __init__(self, tickers):
        self.tickers = tickers
        self.ticker_price = TickerPrice()

    def process_tickers(self):
        """Trains, saves, and predicts for multiple tickers."""
        for ticker in self.tickers:
            print(f"Processing {ticker}...")

            df = self.ticker_price.get_share_prices(tickers=[ticker])
            if df is None or df.empty:
                print(f"No data found for {ticker}. Skipping...")
                continue

            predictor = StockPricePredictor(df, ticker)
            X_train, y_train, X_test, y_test, X_val, y_val = predictor.preprocess_data()

            predictor.train_model(X_train, y_train, X_val, y_val)

            # Evaluate model performance
            predictor.evaluate_model(X_test, y_test)

            
            predictor.save_model()

            next_day_price = predictor.predict_next_day()
            print(f"Next day's predicted price for {ticker}: {next_day_price:.2f}")

            future_prices = predictor.predict_multiple_days(2)
            print(f"Predicted prices for the next 2 days for {ticker}: {future_prices}")



In [2]:
tickers = ['BMW.DE']  # List of tickers to process
manager = StockModelManager(tickers)
manager.process_tickers() 

Processing BMW.DE...
Epoch 1/20


c:\Users\Wambui\Documents\MBD SCHOOL WORK\python_finance_analytics\finance_env\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


27/27 ━━━━━━━━━━━━━━━━━━━━ 3s 30ms/step - loss: 0.0993 - val_loss: 0.0078
Epoch 2/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0088 - val_loss: 0.0032
Epoch 3/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 0.0064 - val_loss: 0.0052
Epoch 4/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0053 - val_loss: 0.0021
Epoch 5/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.0046 - val_loss: 0.0034
Epoch 6/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.0044 - val_loss: 0.0030
Epoch 7/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0042 - val_loss: 0.0020
Epoch 8/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0045 - val_loss: 0.0048
Epoch 9/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0048 - val_loss: 0.0023
Epoch 10/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.0039 - val_loss: 0.0033
Epoch 11/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0040 - val_loss: 0.0016
Epoch 12/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.0041 - val_l

Model Evaluation for BMW.DE:
Mean Squared Error (MSE): 8.6620
Root Mean Squared Error (RMSE): 2.9431
Model saved to models\lstm_model_BMW.DE.h5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Next day's predicted price for BMW.DE: 117.48
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Predicted prices for the next 2 days for BMW.DE: [np.float64(117.47916222214698), np.float64(117.380469789505)]


In [3]:
tickers = ['VOW.DE']  # List of tickers to process
manager = StockModelManager(tickers)
manager.process_tickers()

Processing VOW.DE...
Epoch 1/20


c:\Users\Wambui\Documents\MBD SCHOOL WORK\python_finance_analytics\finance_env\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


27/27 ━━━━━━━━━━━━━━━━━━━━ 3s 33ms/step - loss: 0.0988 - val_loss: 0.0057
Epoch 2/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 0.0084 - val_loss: 0.0015
Epoch 3/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 0.0062 - val_loss: 8.0045e-04
Epoch 4/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.0055 - val_loss: 5.3973e-04
Epoch 5/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0049 - val_loss: 4.8426e-04
Epoch 6/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0046 - val_loss: 4.8950e-04
Epoch 7/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.0044 - val_loss: 3.6423e-04
Epoch 8/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 0.0048 - val_loss: 7.6063e-04
Epoch 9/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 0.0045 - val_loss: 3.7799e-04
Epoch 10/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0041 - val_loss: 4.1123e-04
Epoch 11/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 0.0033 - val_loss: 5.3767e-04
Epoch 12/20
27/27 ━━━━━━━━━━━━━━━━━━━━

Model Evaluation for VOW.DE:
Mean Squared Error (MSE): 73.4717
Root Mean Squared Error (RMSE): 8.5716
Model saved to models\lstm_model_VOW.DE.h5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Next day's predicted price for VOW.DE: 156.33
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Predicted prices for the next 2 days for VOW.DE: [np.float64(156.32787381261588), np.float64(155.9766668304801)]


In [5]:
tickers = ['MBG.DE']  # List of tickers to process
manager = StockModelManager(tickers)
manager.process_tickers()

Processing MBG.DE...
Epoch 1/20


c:\Users\Wambui\Documents\MBD SCHOOL WORK\python_finance_analytics\finance_env\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


27/27 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - loss: 0.1441 - val_loss: 0.0047
Epoch 2/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 0.0092 - val_loss: 0.0022
Epoch 3/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 0.0076 - val_loss: 0.0020
Epoch 4/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.0065 - val_loss: 0.0018
Epoch 5/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.0060 - val_loss: 0.0027
Epoch 6/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.0061 - val_loss: 0.0020
Epoch 7/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.0059 - val_loss: 0.0024
Epoch 8/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 0.0064 - val_loss: 0.0018
Epoch 9/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - loss: 0.0053 - val_loss: 0.0015
Epoch 10/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - loss: 0.0049 - val_loss: 0.0021
Epoch 11/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 0.0048 - val_loss: 0.0019
Epoch 12/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.0047 - val_l

Model Evaluation for MBG.DE:
Mean Squared Error (MSE): 7.5149
Root Mean Squared Error (RMSE): 2.7413
Model saved to models\lstm_model_MBG.DE.h5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Next day's predicted price for MBG.DE: 77.62
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Predicted prices for the next 2 days for MBG.DE: [np.float64(77.62067045331001), np.float64(77.52254195570946)]


In [6]:
tickers = ['FRE.DE']  # List of tickers to process
manager = StockModelManager(tickers)
manager.process_tickers()

Processing FRE.DE...
Epoch 1/20


c:\Users\Wambui\Documents\MBD SCHOOL WORK\python_finance_analytics\finance_env\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


27/27 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - loss: 0.1438 - val_loss: 0.0133
Epoch 2/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 0.0145 - val_loss: 0.0035
Epoch 3/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.0101 - val_loss: 0.0018
Epoch 4/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.0084 - val_loss: 0.0015
Epoch 5/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.0080 - val_loss: 0.0013
Epoch 6/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.0076 - val_loss: 0.0013
Epoch 7/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 0.0068 - val_loss: 0.0012
Epoch 8/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.0069 - val_loss: 0.0012
Epoch 9/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.0064 - val_loss: 0.0011
Epoch 10/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.0060 - val_loss: 0.0013
Epoch 11/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.0067 - val_loss: 0.0011
Epoch 12/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.0057 - val_l

Model Evaluation for FRE.DE:
Mean Squared Error (MSE): 2.2387
Root Mean Squared Error (RMSE): 1.4962
Model saved to models\lstm_model_FRE.DE.h5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Next day's predicted price for FRE.DE: 27.01
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Predicted prices for the next 2 days for FRE.DE: [np.float64(27.01140130996704), np.float64(26.949243240356445)]


In [7]:
tickers = ['BAYN.DE']  # List of tickers to process
manager = StockModelManager(tickers)
manager.process_tickers()

Processing BAYN.DE...
Epoch 1/20


c:\Users\Wambui\Documents\MBD SCHOOL WORK\python_finance_analytics\finance_env\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


27/27 ━━━━━━━━━━━━━━━━━━━━ 3s 37ms/step - loss: 0.1782 - val_loss: 0.0132
Epoch 2/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.0097 - val_loss: 0.0146
Epoch 3/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 0.0083 - val_loss: 0.0103
Epoch 4/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - loss: 0.0075 - val_loss: 0.0072
Epoch 5/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 0.0066 - val_loss: 0.0049
Epoch 6/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 0.0060 - val_loss: 0.0041
Epoch 7/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 0.0057 - val_loss: 0.0037
Epoch 8/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.0058 - val_loss: 0.0028
Epoch 9/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 0.0055 - val_loss: 0.0029
Epoch 10/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 0.0052 - val_loss: 0.0032
Epoch 11/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 0.0057 - val_loss: 0.0035
Epoch 12/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.0052 - val_l

Model Evaluation for BAYN.DE:
Mean Squared Error (MSE): 4.1449
Root Mean Squared Error (RMSE): 2.0359
Model saved to models\lstm_model_BAYN.DE.h5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Next day's predicted price for BAYN.DE: 32.54
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Predicted prices for the next 2 days for BAYN.DE: [np.float64(32.535289154052734), np.float64(32.50909007310867)]
